In [160]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from sklearn.model_selection import train_test_split

# Load training dataset and testing dataset
df = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [161]:
def preprocessing(df):
    
    y = np.zeros((df.shape[0], 26))
    y_next = np.zeros((df.shape[0], 128))
    y_prev = np.zeros((df.shape[0], 128))
    p = np.zeros((df.shape[0], 16))

    for i in range(df.shape[0]):
        y_tmp = np.zeros((1, 26))
        p_tmp = np.zeros((1, 16))
        y_tmp[0, ord(df.iloc[i, 1])-97] = 1
        p_tmp[0, df.iloc[i, 3]] = 1
        y[i, :] = y_tmp
        p[i, :] = p_tmp
        if df.iloc[i, 2] != -1:
            #y_next_tmp=np.zeros((1, 32))
            #y_next_tmp[0, ord(df.iloc[i+1, 1])-97] = 1
            y_next[i, :] = df.iloc[i+1, 4:]
        
        if i > 0 and df.iloc[i-1, 2] != -1:
            #y_prev_tmp=np.zeros((1, 32))
            #y_prev_tmp[0, ord(df.iloc[i-1, 1])-97] = 1
            y_prev[i, :] = df.iloc[i-1, 4:]
        
    return y, y_next, y_prev, p

In [162]:
y, y_next, y_prev, p = preprocessing(df)

In [163]:
y_t, y_t_next, y_t_prev, p_t = preprocessing(df_test)

In [164]:
def preprocessing_2(X, p, y_next, y_prev):
    X = X.drop(columns = ['Id','Prediction', 'NextId', 'Position'])
    X = X.values
    X = np.concatenate((y_prev,X), axis = 1)
    X = np.concatenate((X, y_next), axis = 1)
    X = np.concatenate((X, p), axis = 1)
    X = X.reshape(X.shape[0], 50, 8, 1)
    X = X.astype('float32')
    return X

In [165]:
X_train, X_test, y_train, y_test, p_train, p_test, y_next_train, y_next_test, y_prev_train, y_prev_test = train_test_split(df, y, p, y_next, y_prev, test_size=0.2)

In [166]:
X_train = preprocessing_2(X_train, p_train, y_next_train, y_prev_train)

In [167]:
X_test = preprocessing_2(X_test, p_test, y_next_test, y_prev_test)

In [168]:
# Prepare test data for prediction
final_test = preprocessing_2(df_test, p_t, y_t_next, y_t_prev)

In [169]:
def loadModel():
    # Define model architecture
    model = Sequential()
    model.add(Conv2D(32, 3, 3, activation='relu', input_shape = (50, 8, 1)))
    #model.add(Conv2D(32, 3, 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation ='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(26, activation='softmax'))
    return model

In [170]:
model = loadModel()
    # Compile model
model.compile(loss = 'categorical_crossentropy',
    optimizer = keras.optimizers.adam(),
    metrics = ['accuracy'])
model.fit(X_train, y_train, 
          batch_size =400, epochs=100, verbose = 2)

score = model.evaluate(X_test, y_test, verbose = 0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(50, 8, 1))`
  after removing the cwd from sys.path.


Epoch 1/100
 - 24s - loss: 1.8761 - acc: 0.4688
Epoch 2/100
 - 25s - loss: 0.9477 - acc: 0.7148
Epoch 3/100
 - 22s - loss: 0.6955 - acc: 0.7871
Epoch 4/100
 - 22s - loss: 0.5733 - acc: 0.8246
Epoch 5/100
 - 22s - loss: 0.4908 - acc: 0.8489
Epoch 6/100
 - 22s - loss: 0.4403 - acc: 0.8622
Epoch 7/100
 - 22s - loss: 0.3907 - acc: 0.8766
Epoch 8/100
 - 22s - loss: 0.3591 - acc: 0.8872
Epoch 9/100
 - 22s - loss: 0.3431 - acc: 0.8904
Epoch 10/100
 - 22s - loss: 0.3228 - acc: 0.8962
Epoch 11/100
 - 21s - loss: 0.2986 - acc: 0.9047
Epoch 12/100
 - 22s - loss: 0.2874 - acc: 0.9068
Epoch 13/100
 - 22s - loss: 0.2714 - acc: 0.9123
Epoch 14/100
 - 22s - loss: 0.2635 - acc: 0.9138
Epoch 15/100
 - 22s - loss: 0.2430 - acc: 0.9201
Epoch 16/100
 - 22s - loss: 0.2403 - acc: 0.9222
Epoch 17/100
 - 22s - loss: 0.2325 - acc: 0.9232
Epoch 18/100
 - 22s - loss: 0.2177 - acc: 0.9284
Epoch 19/100
 - 22s - loss: 0.2154 - acc: 0.9266
Epoch 20/100
 - 22s - loss: 0.2077 - acc: 0.9300
Epoch 21/100
 - 22s - loss: 0

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [1.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [171]:
y_labels = model.predict(final_test)

In [172]:
def write_to_csv(list_char):
    
    #index = np.arange(len(list_char))
    #df1 = pd.DataFrame({'Id': index })
    df = pd.DataFrame({'Prediction':list_char })
    
    #df = pd.concat([df1, df2], axis=1)
    df.to_csv('./ZhengShuangyue-A0178479E.csv')

In [174]:
list_result = y_labels.tolist()
list_char = []

for k in range(len(list_result)):
    pred_char = chr(list_result[k].index(max(list_result[k])) + 97)
    list_char.append(pred_char)

write_to_csv(list_char)
# rows = zip(Z_id, output_char)
  

(33254, 26)
(8314, 26)
(33254, 208)
(8314, 208)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(26, 8, 1))`
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.


Epoch 1/100
 - 11s - loss: 1.7789 - acc: 0.4968
Epoch 2/100
 - 10s - loss: 0.7748 - acc: 0.7720
Epoch 3/100
 - 10s - loss: 0.5635 - acc: 0.8328
Epoch 4/100
 - 10s - loss: 0.4714 - acc: 0.8577
Epoch 5/100
 - 10s - loss: 0.4142 - acc: 0.8756
Epoch 6/100
 - 10s - loss: 0.3662 - acc: 0.8880
Epoch 7/100
 - 10s - loss: 0.3408 - acc: 0.8964
Epoch 8/100
 - 10s - loss: 0.3135 - acc: 0.9037
Epoch 9/100
 - 10s - loss: 0.2925 - acc: 0.9075
Epoch 10/100
 - 10s - loss: 0.2703 - acc: 0.9170
Epoch 11/100
 - 10s - loss: 0.2555 - acc: 0.9201
Epoch 12/100
 - 10s - loss: 0.2418 - acc: 0.9240
Epoch 13/100
 - 10s - loss: 0.2307 - acc: 0.9287
Epoch 14/100
 - 10s - loss: 0.2122 - acc: 0.9328
Epoch 15/100
 - 10s - loss: 0.2029 - acc: 0.9354
Epoch 16/100
 - 10s - loss: 0.1916 - acc: 0.9407
Epoch 17/100
 - 10s - loss: 0.1888 - acc: 0.9391
Epoch 18/100
 - 10s - loss: 0.1820 - acc: 0.9432
Epoch 19/100
 - 10s - loss: 0.1740 - acc: 0.9431
Epoch 20/100
 - 10s - loss: 0.1638 - acc: 0.9483
Epoch 21/100
 - 10s - loss: 0

TypeError: evaluate() got an unexpected keyword argument 'steps_per_epoch'

In [ ]:
# Predict test dataset

